# Data files:
* Kaggle data (df_2_kaggle.csv)
* activity data (parks_activity_data.json)
* data.world data (us-national-parks-dataset.json)

In [1]:
# Dependencies
import pandas as pd
import json
import csv
import pymongo
from pymongo import MongoClient

### Kaggle data (df_2_kaggle.csv)

In [2]:
#  Extract CSV file into dataframe
kaggle_df = pd.read_csv('Resources/df_2_kaggle.csv')
# kaggle_df.head()
# Data cleaning: remove first unnamed column
del kaggle_df['Unnamed: 0']
kaggle_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/df_2_kaggle.csv'

In [ ]:
# Convert dataframe to dictinary
dict_data=kaggle_df.to_dict("records")
dict_data


[{'Name': 'Acadia',
  'Image': nan,
  'Location': 'Maine.mw-parser-output .geo-default,.mw-parser-output .geo-dms,.mw-parser-output .geo-dec{display:inline}.mw-parser-output .geo-nondefault,.mw-parser-output .geo-multi-punct{display:none}.mw-parser-output .longitude,.mw-parser-output .latitude{white-space:nowrap}44°21′N 68°13′W\ufeff / \ufeff44.35°N 68.21°W',
  'Date established as park[7][12]': 'February 26, 1919',
  'Area (2021)[13]': '49,071.40 acres (198.6\xa0km2)',
  'Recreation visitors (2021)[11]': 4069098,
  'Description': 'Covering most of Mount Desert Island and other coastal islands, Acadia features the tallest mountain on the Atlantic coast of the United States, granite peaks, ocean shoreline, woodlands, and lakes. There are freshwater, estuary, forest, and intertidal habitats.[14][15]'},
 {'Name': 'American Samoa',
  'Image': nan,
  'Location': 'American Samoa14°15′S 170°41′W\ufeff / \ufeff14.25°S 170.68°W',
  'Date established as park[7][12]': 'October 31, 1988',
  'Area 

In [ ]:
# create a table and collection in mongoDB 
#drop the data into the database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.NationalTreasure_db
#drop the collection if exists
db.kaggle.drop()

#collection 

collection = db['kaggle']

# Insert collection
collection.insert_many(dict_data)

### activity data (parks_activity_data.json)

In [ ]:
# Read JSON file instead of API call
f = open('Resources/parks_activity_data.json')
results = json.load(f)
results

{'total': '40',
 'limit': '50',
 'start': '0',
 'data': [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42',
   'name': 'Arts and Culture',
   'parks': [{'states': 'ME',
     'parkCode': 'acad',
     'designation': 'National Park',
     'fullName': 'Acadia National Park',
     'url': 'https://www.nps.gov/acad/index.htm',
     'name': 'Acadia'},
    {'states': 'NY',
     'parkCode': 'afbg',
     'designation': 'National Monument',
     'fullName': 'African Burial Ground National Monument',
     'url': 'https://www.nps.gov/afbg/index.htm',
     'name': 'African Burial Ground'},
    {'states': 'NE',
     'parkCode': 'agfo',
     'designation': 'National Monument',
     'fullName': 'Agate Fossil Beds National Monument',
     'url': 'https://www.nps.gov/agfo/index.htm',
     'name': 'Agate Fossil Beds'},
    {'states': 'TX',
     'parkCode': 'alfl',
     'designation': 'National Monument',
     'fullName': 'Alibates Flint Quarries National Monument',
     'url': 'https://www.nps.gov/alfl/index.h

In [ ]:
# Extract activity data from results (non-pandas way)

# create an empty list (will become df later)
data_list = []

# use a for loop to go through results data

for i in results['data']:
    #print(i)
    act_id = i['id']
    act_name = i['name']
    parks_list = [park_dict['fullName'] for park_dict in i['parks']]
    for park_dict in i['parks']:
        park_name = park_dict['fullName']
        states = park_dict['states']  # extract the states
        df2 = {
            'act_name': act_name,
            'park_name': park_name,
            'states': states  # add the states to the dictionary
        }
        data_list.append(df2)

# convert into dataframe
activity_df = pd.DataFrame(data_list)
activity_df

,act_name,park_name,states
0,Arts and Culture,Acadia National Park,ME
1,Arts and Culture,African Burial Ground National Monument,NY
2,Arts and Culture,Agate Fossil Beds National Monument,NE
3,Arts and Culture,Alibates Flint Quarries National Monument,TX
4,Arts and Culture,Ala Kahakai National Historic Trail,HI
...,...,...,...
3935,Wildlife Watching,Yellowstone National Park,"ID,MT,WY"
3936,Wildlife Watching,Yorktown Battlefield Part of Colonial National...,VA
3937,Wildlife Watching,Yosemite National Park,CA
3938,Wildlife Watching,Yukon - Charley Rivers National Preserve,AK


In [ ]:
# Reorganize data --> key: park_name; value: list of corresponding activities (act_name)
# act_dict = activity_df.groupby('park_name')[['act_name','states']].apply(list).to_dict()
# act_dict
act_dict = activity_df.groupby('park_name')[['act_name','states']].apply(lambda g: g.values.tolist()).to_dict()
# act_dict
act_new_list = []
for k in list(act_dict.keys()):
#     print(k)
#     print(act_dict[k][0][1])
    act_tem = []
    for act in act_dict[k]:
#         print(act[0])
        act_tem.append(act[0])
#     print(act_tem)
    state = act_dict[k][0][1]
    act_new_df = {
        'park_name':k,
        'act_name':act_tem,
        'states': state
    }

    act_new_list.append(act_new_df)
    
activity_new_df = pd.DataFrame(act_new_list)
activity_new_df.head()

,park_name,act_name,states
0,**********National Mall**********,"[Arts and Culture, Boating, Food, Guided Tours...",DC
1,Abraham Lincoln Birthplace National Historical...,"[Astronomy, Food, Guided Tours, Junior Ranger ...",KY
2,Acadia National Park,"[Arts and Culture, Astronomy, Biking, Boating,...",ME
3,Adams National Historical Park,"[Guided Tours, Living History, Park Film, Shop...",MA
4,African American Civil War Memorial,[Guided Tours],DC


In [ ]:
# Convert dataframe to dictinary
dict_data=activity_new_df.to_dict("records")
dict_data


[{'park_name': '**********National Mall**********',
  'act_name': ['Arts and Culture',
   'Boating',
   'Food',
   'Guided Tours',
   'Shopping',
   'Wildlife Watching'],
  'states': 'DC'},
 {'park_name': 'Abraham Lincoln Birthplace National Historical Park',
  'act_name': ['Astronomy',
   'Food',
   'Guided Tours',
   'Junior Ranger Program',
   'Museum Exhibits',
   'Park Film',
   'Shopping',
   'Wildlife Watching'],
  'states': 'KY'},
 {'park_name': 'Acadia National Park',
  'act_name': ['Arts and Culture',
   'Astronomy',
   'Biking',
   'Boating',
   'Camping',
   'Climbing',
   'Compass and GPS',
   'Fishing',
   'Food',
   'Guided Tours',
   'Hands-On',
   'Hiking',
   'Horse Trekking',
   'Ice Skating',
   'Junior Ranger Program',
   'Paddling',
   'Park Film',
   'Shopping',
   'Skiing',
   'Snow Play',
   'Snowmobiling',
   'Snowshoeing',
   'Swimming',
   'Wildlife Watching'],
  'states': 'ME'},
 {'park_name': 'Adams National Historical Park',
  'act_name': ['Guided Tours',

In [ ]:
# create a table and collection in mongoDB 
#drop the data into the database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.NationalTreasure_db
#drop the collection if exists
db.activity.drop()

#collection 

collection = db['activity']

# Insert collection
collection.insert_many(dict_data)

### data.world data (us-national-parks-dataset.json)

In [ ]:
#  Extract JSON file into DataFrame
dataworld_df = pd.read_json('Resources/us-national-parks-dataset.json')
dataworld_df.head()

,area,coordinates,date_established_readable,date_established_unix,description,image,nps_link,states,title,id,visitors,world_heritage_site
0,"{'acres': '49,057.36', 'square_km': '198.5'}","{'latitude': 44.35, 'longitude': -68.21}","February 26, 1919",-1604599200,Covering most of Mount Desert Island and other...,"{'url': 'acadia.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/acad/index.htm,"[{'id': 'state_maine', 'title': 'Maine'}]",Acadia,park_acadia,"3,303,393",False
1,"{'acres': '8,256.67', 'square_km': '33.4'}","{'latitude': -14.25, 'longitude': -170.68}","October 31, 1988",594280800,The southernmost National Park is on three Sam...,"{'url': 'american-samoa.jpg', 'attribution': '...",https://www.nps.gov/npsa/index.htm,"[{'id': 'state_american-samoa', 'title': 'Amer...",American Samoa,park_american-samoa,"28,892",False
2,"{'acres': '76,678.98', 'square_km': '310.3'}","{'latitude': 38.68, 'longitude': -109.57}","November 12, 1971",58773600,"This site features more than 2,000 natural san...","{'url': 'arches.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/arch/index.htm,"[{'id': 'state_utah', 'title': 'Utah'}]",Arches,park_arches,"1,585,718",False
3,"{'acres': '242,755.94', 'square_km': '982.4'}","{'latitude': 43.75, 'longitude': -102.5}","November 10, 1978",279525600,"The Badlands are a collection of buttes, pinna...","{'url': 'badlands.jpg', 'attribution': 'PixelB...",https://www.nps.gov/badl/index.htm,"[{'id': 'state_south-dakota', 'title': 'South ...",Badlands,park_badlands,"996,263",False
4,"{'acres': '801,163.21', 'square_km': '3,242.2'}","{'latitude': 29.25, 'longitude': -103.25}","June 12, 1944",-806439600,Named for the prominent bend in the Rio Grande...,"{'url': 'big-bend.jpg', 'attribution': 'PixelB...",https://www.nps.gov/bibe/index.htm,"[{'id': 'state_texas', 'title': 'Texas'}]",Big Bend,park_big-bend,"388,290",False


In [ ]:
dict_data=dataworld_df.to_dict("records")
dict_data

[{'area': {'acres': '49,057.36', 'square_km': '198.5'},
  'coordinates': {'latitude': 44.35, 'longitude': -68.21},
  'date_established_readable': 'February 26, 1919',
  'date_established_unix': -1604599200,
  'description': 'Covering most of Mount Desert Island and other coastal islands, Acadia features the tallest mountain on the Atlantic coast of the United States, granite peaks, ocean shoreline, woodlands, and lakes. There are freshwater, estuary, forest, and intertidal habitats.',
  'image': {'url': 'acadia.jpg',
   'attribution': 'PixelBay/@Skeeze',
   'attribution_url': 'https://pixabay.com/en/users/skeeze-272447/'},
  'nps_link': 'https://www.nps.gov/acad/index.htm',
  'states': [{'id': 'state_maine', 'title': 'Maine'}],
  'title': 'Acadia',
  'id': 'park_acadia',
  'visitors': '3,303,393',
  'world_heritage_site': False},
 {'area': {'acres': '8,256.67', 'square_km': '33.4'},
  'coordinates': {'latitude': -14.25, 'longitude': -170.68},
  'date_established_readable': 'October 31,

In [ ]:
# create a table and collection in mongoDB 
#drop the data into the database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.NationalTreasure_db
#drop the collection if exists
db.dataworld.drop()

#collection 

collection = db['dataworld']

# Insert collection
collection.insert_many(dict_data)